In [58]:
!pip install psycopg2-binary gspread gspread-dataframe pandas oauth2client ipykernel

In [59]:
import psycopg2
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import json
import os

In [60]:
# --- Celda de Autenticación y Secretos ---

# 1. Autenticación con Google
if 'GCP_SA_KEY' in os.environ:
    print("Autenticando con Cuenta de Servicio desde GitHub Secrets...")
    keyfile_dict = json.loads(os.environ['GCP_SA_KEY'])
    gc = gspread.service_account_from_dict(keyfile_dict)
else:
    print("Autenticando con usuario de Colab...")
    from google.colab import auth
    from google.auth import default
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

# 2. Obtención de la Contraseña de la Base de Datos
if 'DB_PASSWORD' in os.environ:
    db_password = os.environ['DB_PASSWORD']
    print("Contraseña de BBDD leída desde GitHub Secrets.")
else:
    try:
        from google.colab import userdata
        db_password = userdata.get('DB_PASSWORD')
        print("Contraseña de BBDD leída desde Colab Secrets.")
    except ImportError:
        db_password = "TU_CONTRASEÑA_AQUÍ" # <-- Solo para pruebas locales si Colab Secrets falla
        print("ADVERTENCIA: Usando contraseña local del código.")

Autenticando con usuario de Colab...
Contraseña de BBDD leída desde Colab Secrets.


In [61]:
# --------------- CONFIGURACIÓN FINAL ---------------
db_host = 'ec2-52-206-224-222.compute-1.amazonaws.com'
db_name = 'd77mgnf951v3g0'
db_user = 'reports'
sheet_name = 'Prueba automatización COLAB' # <-- O el nombre final que le des a tu hoja
sql_query = """
    SELECT
      s.terms_accepted_at::date AS fecha_firma,
      COUNT(DISTINCT s.id) AS cantidad_de_firmas
    FROM
      stores s
    WHERE
      s.terms_accepted_at >= CURRENT_DATE - INTERVAL '365 days'
    GROUP BY
      fecha_firma
    ORDER BY
      fecha_firma DESC
"""
# ----------------------------------------------------

print("--- INICIANDO PROCESO ---")
conn = None

try:
    # 1. Conexión a la Base de Datos
    print("Checkpoint 1: Conectando a PostgreSQL...")
    conn = psycopg2.connect(
        host=db_host,
        dbname=db_name,
        user=db_user,
        password=db_password,
        port=5432
    )
    print("Checkpoint 2: Conexión a BBDD exitosa.")

    # 2. Ejecución de la Query
    print("Checkpoint 3: Ejecutando query...")
    df = pd.read_sql_query(sql_query, conn)
    print(f"Checkpoint 4: Query ejecutada. Se encontraron {len(df)} filas.")

    if len(df) == 0:
        print("ADVERTENCIA: La consulta no devolvió filas. La hoja de cálculo quedará en blanco.")

    # 3. Conexión con Google Sheets
    print(f"Checkpoint 5: Abriendo Google Sheet: '{sheet_name}'...")
    spreadsheet = gc.open(sheet_name)
    print("Checkpoint 6: Hoja de cálculo abierta.")

    worksheet = spreadsheet.get_worksheet(0)
    print(f"Checkpoint 7: Pestaña '{worksheet.title}' seleccionada.")

    print("Checkpoint 8: Limpiando la hoja y escribiendo datos...")
    worksheet.clear()

    # Solo escribe si el dataframe no está vacío
    if not df.empty:
      set_with_dataframe(worksheet, df)

    print("Checkpoint 9: Escritura de datos finalizada.")

    print("\n--- ¡PROCESO COMPLETADO CON ÉXITO! ---")

except Exception as e:
    print(f"\n--- ERROR CRÍTICO DETECTADO ---")
    print(f"Detalle del error: {e}")
    raise e

finally:
    if conn is not None:
        conn.close()
        print("Conexión a la base de datos cerrada.")

--- INICIANDO PROCESO ---
Checkpoint 1: Conectando a PostgreSQL...
Checkpoint 2: Conexión a BBDD exitosa.
Checkpoint 3: Ejecutando query...


/tmp/ipython-input-1076343499.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Checkpoint 4: Query ejecutada. Se encontraron 322 filas.
Checkpoint 5: Abriendo Google Sheet: 'Prueba automatización COLAB'...
Checkpoint 6: Hoja de cálculo abierta.
Checkpoint 7: Pestaña 'Hoja 1' seleccionada.
Checkpoint 8: Limpiando la hoja y escribiendo datos...
Checkpoint 9: Escritura de datos finalizada.

--- ¡PROCESO COMPLETADO CON ÉXITO! ---
Conexión a la base de datos cerrada.
